In [2]:
from manim import *

Manim Community v0.18.0

In [41]:
class PolygonOnAxes(Scene):
    def get_rectangle_corners(self, bottom_left, top_right):
        return [
            (top_right[0], top_right[1]),
            (bottom_left[0], top_right[1]),
            (bottom_left[0], bottom_left[1]),
            (top_right[0], bottom_left[1]),
        ]

    def construct(self):
        ax = Axes(
            x_range=[0, 10],
            y_range=[0, 10],
            x_length=6,
            y_length=6,
            axis_config={"include_tip": False},
        )

        t = ValueTracker(5)
        k = 25

        graph = ax.plot(
            lambda x: k / x,
            color=YELLOW_D,
            x_range=[k / 10, 10.0, 0.01],
            use_smoothing=False,
        )

        def get_rectangle():
            polygon = Polygon(
                *[
                    ax.c2p(*i)
                    for i in self.get_rectangle_corners(
                        (0, 0), (t.get_value(), k / t.get_value())
                    )
                ]
            )
            polygon.stroke_width = 1
            polygon.set_fill(BLUE, opacity=0.5)
            polygon.set_stroke(YELLOW_B)
            return polygon

        polygon = always_redraw(get_rectangle)

        dot = Dot()
        dot.add_updater(lambda x: x.move_to(ax.c2p(t.get_value(), k / t.get_value())))
        dot.set_z_index(10)

        self.add(ax, graph, dot)
        self.play(Create(polygon))
        self.play(t.animate.set_value(10))
        self.play(t.animate.set_value(k / 10))
        self.play(t.animate.set_value(5))


%manim -qm -v WARNING PolygonOnAxes

In [48]:
class SineCurveUnitCircle(Scene):
    # contributed by heejin_park, https://infograph.tistory.com/230
    def construct(self):
        self.show_axis()
        self.show_circle()
        self.move_dot_and_draw_curve()
        self.wait()

    def show_axis(self):
        x_start = np.array([-6,0,0])
        x_end = np.array([6,0,0])

        y_start = np.array([-4,-2,0])
        y_end = np.array([-4,2,0])

        x_axis = Line(x_start, x_end)
        y_axis = Line(y_start, y_end)

        self.add(x_axis, y_axis)
        self.add_x_labels()

        self.origin_point = np.array([-4,0,0])
        self.curve_start = np.array([-3,0,0])

    def add_x_labels(self):
        x_labels = [
            MathTex("\pi"), MathTex("2 \pi"),
            MathTex("3 \pi"), MathTex("4 \pi"),
        ]

        for i in range(len(x_labels)):
            x_labels[i].next_to(np.array([-1 + 2*i, 0, 0]), DOWN)
            self.add(x_labels[i])

    def show_circle(self):
        circle = Circle(radius=1)
        circle.move_to(self.origin_point)
        self.add(circle)
        self.circle = circle

    def create_angle_marker(self, angle, radius=0.5, color=RED):
        arc = Arc(radius=radius, start_angle=0, angle=angle, color=color)
        arc.move_arc_center_to(self.origin_point)
        label = MathTex(f"{angle / PI :.2g} \\pi")
        label.next_to(arc, DOWN, buff=0.1)
        return arc, label

    def move_dot_and_draw_curve(self):
        orbit = self.circle
        origin_point = self.origin_point

        dot = Dot(radius=0.08, color=YELLOW)
        dot.move_to(orbit.point_from_proportion(0))
        self.t_offset = 0
        rate = 0.25

        def go_around_circle(mob, dt):
            self.t_offset += (dt * rate)
            # print(self.t_offset)
            mob.move_to(orbit.point_from_proportion(self.t_offset % 1))

        def get_line_to_circle():
            return Line(origin_point, dot.get_center(), color=BLUE)

        def get_line_to_curve():
            x = self.curve_start[0] + self.t_offset * 4
            y = dot.get_center()[1]
            return Line(dot.get_center(), np.array([x,y,0]), color=YELLOW_A, stroke_width=2 )
        
        def update_angle_marker(self, *args, **kwargs):
            angle = self.t_offset * TAU
            if angle > TAU:
                angle = angle % TAU
            
            for quarter in [0, PI/2, PI, 3*PI/2, 2*PI, 5*PI/2, 3*PI, 7*PI/2, 4*PI]:
                if np.isclose(angle, quarter, atol=0.1):
                    arc, label = self.create_angle_marker(quarter)
                    self.add(arc, label)
            # arc, label = self.create_angle_marker(angle)
            # return VGroup(arc, label)
        

        self.add_updater(update_angle_marker)
        # angle_marker = always_redraw(update_angle_marker)
        # self.add(angle_marker)
        self.remove_updater(update_angle_marker)


        self.curve = VGroup()
        self.curve.add(Line(self.curve_start,self.curve_start))
        def get_curve():
            last_line = self.curve[-1]
            x = self.curve_start[0] + self.t_offset * 4
            y = dot.get_center()[1]
            new_line = Line(last_line.get_end(),np.array([x,y,0]), color=YELLOW_D)
            self.curve.add(new_line)

            return self.curve

        dot.add_updater(go_around_circle)

        origin_to_circle_line = always_redraw(get_line_to_circle)
        dot_to_curve_line = always_redraw(get_line_to_curve)
        sine_curve_line = always_redraw(get_curve)

        self.add(dot)
        self.add(orbit, origin_to_circle_line, dot_to_curve_line, sine_curve_line)
        self.wait(8.5)

        dot.remove_updater(go_around_circle)

%manim -qm -v WARNING SineCurveUnitCircle

In [35]:
class QuadraticFormula(Scene):
    def construct(self):
        # Writing the general form of a quadratic equation
        equation_1 = MathTex("ax^2", "+", "bx", "+", "c", "=", "0").scale(1.5)
        self.play(Write(equation_1))
        self.wait(1)

        # Create the components of the new equation
        equation_2 = MathTex("x^2", "+", "\\frac{b}{a}x", "+", "\\frac{c}{a}", "=", "0").scale(1.5)

        # Animation for transforming each part of the equation
        self.play(
            ReplacementTransform(equation_1, equation_2),  # Transform all
            
        )
        self.wait(1)

        
        


%manim -qm -v WARNING QuadraticFormula

# Note: the default grid size is 8 x 14. It's addressed by a numpy array with x=[-7,8], y = [-4:5] and (0,0) is the centre

In [38]:
class QuadraticFormula(Scene):
    def construct(self):
        # Writing the general form of a quadratic equation
        equation_1 = MathTex("a", "x^2", "+", "b", "x", "+", "c", "=", "0").scale(1.5)
        self.play(Write(equation_1))
        self.wait(1)

        # Highlight the first coefficient "a"
        self.play(ApplyMethod(equation_1[0].set_color, RED))
        self.wait(1)

        # Divide by 'a' to simplify
        equation_2 = MathTex(
            "x^2", "+", 
            "\\frac{b}{a}", "x", "+", 
            "\\frac{c}{a}", 
            "=", "0"
        ).scale(1.5)

        # Create equation 2 but without the denominator that we'll get from the coefficient of the first term
        equation_2_partial = MathTex(
            "x^2", "+", 
            "\\frac{b}{\\phantom{a}}", "x", "+", 
            "\\frac{c}{\\phantom{a}}", 
            "=", "0"
        ).scale(1.5)

        # manually adjust vertical position of equation_2_partial to that of equation_2
        equation_2_partial.shift(UP * 0.23) # This value may need to be adjusted

        # Duplicate 'a' and move it under 'b' and 'c'
        a_copy1 = equation_1[0].copy()
        a_copy2 = equation_1[0].copy()
        a_copy1.target = MathTex("a").scale(1.5).set_color(RED).move_to(equation_2[2]).align_to(equation_2[2], DOWN) #control the color and alignment
        a_copy2.target = MathTex("a").scale(1.5).set_color(RED).move_to(equation_2[5]).align_to(equation_2[5], DOWN) #control the color and alignment

        self.play(
            MoveToTarget(a_copy1),  # Move first 'a' to its target
            MoveToTarget(a_copy2),   # Move second 'a' to its target
            ReplacementTransform(equation_1, equation_2_partial) # Transform from equation_1 to equation_2_partial
        )


        self.play(
            FadeIn(equation_2),
            FadeOut(a_copy1, a_copy2)
        )
        self.play(
            FadeOut(equation_2_partial)
        )     
               
        self.wait(1)

        # Adjust the representation 

        equation_3 = MathTex(
            "x^2", "+",
            "\\frac{b}{a}x",
            "=",  "-",
            "\\frac{c}{a}"
        ).scale(1.5)

        equation_3_partial = MathTex(
            "x^2", "+",
            "\\frac{b}{a}x",
            "+",
            "{\\phantom{\\frac{c}{a}}}",
            "=", "-",
            "{\\phantom{\\frac{c}{a}}}"
        ).scale(1.5)

        equation_3_partial.shift(RIGHT * 0.1)

        ca_copy = equation_2[6].copy()
        ca_copy.target = MathTex("\\frac{c}{a}").scale(1.5).move_to(equation_3_partial[6])
    
        ca_copy.target.shift(RIGHT * 0.8)

        self.play(
            Unwrite(equation_2[-1])
        )
        self.wait(1)

        self.play(
            MoveToTarget(ca_copy),
            
            FadeOut(equation_2),
            FadeIn(equation_3_partial),
            FadeOut(equation_3_partial[3])
        )

        self.play(
            TransformMatchingTex(equation_3_partial, equation_3),
            
            Transform(ca_copy, equation_3[5]),
            FadeOut(ca_copy)
            
        )

        
        self.wait(1)

        # Complete the square

        # Group the left and right sides of the equation
        left_side = VGroup(equation_3[0], equation_3[1], equation_3[2]) # x^2 + (b/a)x
        right_side = VGroup(equation_3[4], equation_3[5]) # - (c/a)

        # Shift those parts
        self.play(left_side.animate.shift(LEFT * 3.0), right_side.animate.shift(RIGHT * 2.5))

        # Add the square completion term to both sides
        term_left = MathTex("+\\left(\\frac{b}{2a}\\right)^2").scale(1.5).next_to(left_side, RIGHT)
        term_right = MathTex("\\left(\\frac{b}{2a}\\right)^2").scale(1.5).next_to(right_side, LEFT)

        # term_left = new_term.copy().next_to(left_side, RIGHT)
        # term_right = new_term.copy().next_to(right_side, LEFT)

        # animate the terms addition
        self.play(
            Write(term_left),
            Write(term_right)
        )

        self.wait(2)



        

%manim -qm -v WARNING QuadraticFormula

In [18]:
class CreateCircle(Scene):
    def construct(self):
        circle = Circle()  # create a circle
        circle.set_fill(PINK, opacity=0.5)  # set the color and transparency
        self.play(Create(circle))  # show the circle on screen



# don't remove below command for run button to work
%manim -qm -v WARNING CreateCircle



In [23]:
class SquareToCircle(Scene):
    def construct(self):
        circle = Circle()  # create a circle
        circle.set_fill(PINK, opacity=0.5)  # set color and transparency

        square = Square()  # create a square
        square.rotate(PI / 4)  # rotate a certain amount

        self.play(Create(square))  # animate the creation of the square
        self.play(Transform(square, circle))  # interpolate the square into the circle
        self.play(FadeOut(square))  # fade out animation

%manim -qm -v WARNING SquareToCircle

In [28]:
class SquareAndCircle(Scene):
    def construct(self):
        circle = Circle()  # create a circle
        circle.set_fill(PINK, opacity=0.5)  # set the color and transparency

        square = Square()  # create a square
        square.set_fill(BLUE, opacity=0.5)  # set the color and transparency

        square.next_to(circle, RIGHT, buff=0.5)  # set the position
        self.play(FadeIn(circle), FadeIn(square))  # show the shapes on screen
        self.play(FadeOut(square), FadeOut(circle))
        #self.play(FadeOut(circle))

%manim -qm -v WARNING SquareAndCircle

In [30]:
class AnimatedSquareToCircle(Scene):
    def construct(self):
        circle = Circle()  # create a circle
        square = Square()  # create a square

        self.play(Create(square))  # show the square on screen
        self.play(square.animate.rotate(PI / 4))  # rotate the square
        self.play(
            ReplacementTransform(square, circle)
        )  # transform the square into a circle
        self.play(
            circle.animate.set_fill(PINK, opacity=0.5)
        )  # color the circle on screen

class DifferentRotations(Scene):
    def construct(self):
        left_square = Square(color=BLUE, fill_opacity=0.7).shift(2 * LEFT)
        right_square = Square(color=GREEN, fill_opacity=0.7).shift(2 * RIGHT)
        self.play(
            left_square.animate.rotate(PI), Rotate(right_square, angle=PI), run_time=2
        )
        self.wait()

%manim -qm -v WARNING DifferentRotations

In [35]:
class RunTime(Scene):
    def construct(self):
        square = Square()
        self.add(square)
        self.play(square.animate.shift(UP), run_time=1.6)
        self.wait(1)

%manim -qm -v WARNING RunTime

In [47]:
class Count(Animation):
    def __init__(self, number: DecimalNumber, start: float, end: float, **kwargs) -> None:
        # Pass number as the mobject of the animation
        super().__init__(number,  **kwargs)
        # Set start and end
        self.start = start
        self.end = end

    def interpolate_mobject(self, alpha: float) -> None:
        # Set value of DecimalNumber according to alpha
        value = self.start + (alpha * (self.end - self.start))
        self.mobject.set_value(value)


class CountingScene(Scene):
    def construct(self):
        # Create Decimal Number and add it to scene
        number = DecimalNumber().set_color(WHITE).scale(5)
        # Add an updater to keep the DecimalNumber centered as its value changes
        number.add_updater(lambda number: number.move_to(ORIGIN))

        self.add(number)

        self.wait()

        # Play the Count Animation to count from 0 to 100 in 4 seconds
        self.play(Count(number, 0, 100), run_time=4, rate_func=rate_functions.linear)

        self.wait()

%manim -qm -v WARNING CountingScene

In [56]:
class RateFunctions1Example(Scene):
    def construct(self):
        line1 = Line(3*LEFT, 3*RIGHT).shift(UP).set_color(RED)
        line2 = Line(3*LEFT, 3*RIGHT).set_color(GREEN)
        line3 = Line(3*LEFT, 3*RIGHT).shift(DOWN).set_color(BLUE)
        line4 = Line(3*LEFT, 3*RIGHT).shift(2 * DOWN).set_color(YELLOW)

        dot1 = Dot().move_to(line1.get_left())
        dot2 = Dot().move_to(line2.get_left())
        dot3 = Dot().move_to(line3.get_left())
        dot4 = Dot().move_to(line4.get_left())

        label1 = Tex("Ease In").next_to(line1, RIGHT)
        label2 = Tex("Ease out").next_to(line2, RIGHT)
        label3 = Tex("Ease In Out").next_to(line3, RIGHT)
        label4 = Tex("Ease Out Expo").next_to(line4, RIGHT)

        self.play(
            FadeIn(VGroup(line1, line2, line3, line4)),
            FadeIn(VGroup(dot1, dot2, dot3, dot4)),
            Write(VGroup(label1, label2, label3, label4)),
        )
        self.play(
            MoveAlongPath(dot1, line1, rate_func=rate_functions.ease_in_sine),
            MoveAlongPath(dot2, line2, rate_func=rate_functions.ease_out_sine),
            MoveAlongPath(dot3, line3, rate_func=rate_functions.ease_in_out_sine),
            MoveAlongPath(dot4, line4, rate_func=rate_functions.ease_out_expo),
            run_time=7
        )
        self.wait()

%manim -qm -v WARNING RateFunctions1Example

In [59]:
class MovingCameraOnGraph(MovingCameraScene):
    def construct(self):
        self.camera.frame.save_state()

        ax = Axes(x_range=[-1, 20], y_range=[-1, 10])
        graph = ax.plot(lambda x: np.sin(x), color=WHITE, x_range=[0, 6 * PI])

        dot_1 = Dot(ax.i2gp(graph.t_min, graph))
        dot_2 = Dot(ax.i2gp(graph.t_max, graph))
        self.add(ax, graph, dot_1, dot_2)

        self.play(self.camera.frame.animate.scale(0.5).move_to(dot_1))
        self.play(self.camera.frame.animate.move_to(dot_2))
        self.play(Restore(self.camera.frame))
        self.wait()

%manim -qm -v WARNING MovingCameraOnGraph

In [77]:
class MovingCameraOnDot(MovingCameraScene):
    def construct(self):
        self.camera.frame.save_state()

        # Axes and graph definition
        ax = Axes(x_range=[-1, 20], y_range=[-1, 1])
        graph = ax.plot(lambda x: np.sin(x), color=WHITE, x_range=[0, 6 * PI])

        # Moving dot
        moving_dot = Dot(ax.i2gp(0, graph), color=WHITE)

        # Add elements to the scene
        self.add(ax, graph, moving_dot)

        # ValueTracker to control the position of the dot
        dot_position = ValueTracker(0)

        # Custom function for color interpolation
        def color_gradient(y):
            if y >= 0:
                return interpolate_color(WHITE, RED, y)
            else:
                return interpolate_color(WHITE, BLUE, -y)

        # Update function for the moving dot
        def update_dot(mob):
            new_x = dot_position.get_value()
            new_y = np.sin(new_x)
            mob.move_to(ax.i2gp(new_x, graph))
            mob.set_color(color_gradient(new_y))

        # Apply the update function to the moving dot
        moving_dot.add_updater(update_dot)

        # Update function for the camera to follow the dot
        def update_camera(mob):
            mob.move_to(moving_dot.get_center())

        # Start with a wide view of the graph
        self.play(self.camera.frame.animate.set_width(ax.width * 1.2).move_to(ax.get_center()))

        # Zoom into the starting position of the dot
        self.play(self.camera.frame.animate.scale(0.5).move_to(moving_dot))

        # Apply the update function to the camera frame
        self.camera.frame.add_updater(update_camera)

        # Animate the dot movement along with camera panning
        self.play(dot_position.animate.set_value(6 * PI), run_time=8, rate_func=rate_functions.linear)

        # Restore the camera frame and remove updaters
        self.camera.frame.remove_updater(update_camera)
        self.play(Restore(self.camera.frame))
        moving_dot.remove_updater(update_dot)
        self.wait()

%manim -qm -v WARNING MovingCameraOnDot